In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import sqlite3
from time import sleep
import re
import warnings
warnings.filterwarnings("ignore")

### Getting player ids of verified players from Dotabuff

```python
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/'
    '537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
source = requests.get('https://www.dotabuff.com/players', headers=headers)
soup = BeautifulSoup(source.text, 'lxml')
pids = [i['href'].split("/")[2] for i in soup.select('td a.link-type-player')]
```

### Pickup where we left

In [164]:
def pickup_impact(pid_list):
    try:
        pids_impact = pd.read_sql('''SELECT DISTINCT PID FROM Impact''', 
                                  conn)['PID'].unique()
        new_impact = [i for i in pids if i not in pids_impact]   
    except:
        new_impact = pid_list  
    return new_impact

In [10]:
def pickup_economy(pid_list):
    try:
        pids_economy = pd.read_sql('''SELECT DISTINCT PID FROM Economy''', 
                                   conn)['PID'].unique()
        new_economy = [i for i in pids if i not in pids_economy]   
    except:
        new_economy = pid_list  
    return new_economy

In [11]:
def pickup_played(pid_list):
    try:
        pids_played = pd.read_sql('''SELECT DISTINCT PID FROM Played''', 
                                  conn)['PID'].unique()
        new_played = [i for i in pids if i not in pids_played]   
    except:
        new_played = pid_list  
    return new_played

In [12]:
def pickup_itemeco(pid_list):
    try:
        pids_itemeco = pd.read_sql('''SELECT DISTINCT PID FROM ItemEco''', 
                                   conn)['PID'].unique()
        new_itemeco = [i for i in pids if i not in pids_itemeco]   
    except:
        new_itemeco = pid_list  
    return new_itemeco

In [163]:
def pickup_itemmain(pid_list):
    try:
        pids_itemmain = pd.read_sql('''SELECT DISTINCT PID FROM ItemMain''', 
                                    conn)['PID'].unique()
        new_itemmain = [i for i in pids if i not in pids_itemmain]   
    except:
        new_itemmain = pid_list  
    return new_itemmain

## Scraping using Selenium

In [2]:
chrome_path = r'C:\Users\jedda\Downloads\chromedriver.exe'
conn = sqlite3.connect('dota.db')

### Impact

```python
pids_impact = pickup_impact(pids)
driver = webdriver.Chrome(chrome_path)
for p in pids_impact:
    driver.get('https://www.dotabuff.com/players/'+p+'/heroes?metric=impact')
    query = driver.find_elements_by_xpath(
        '/html/body/div[1]/div[8]/div[3]/section/article/table/tbody/*')
    rows = [tuple(i.text.split('\n')) for i in query]
    header = ['Hero', 'Date', 'KDA Ratio', 'Kills', 'Death', 'Assists']
    impactdb = pd.DataFrame(rows, columns=header)
    impactdb.drop(columns='Date', inplace=True)
    impactdb['PID'] = p
    impactdb.to_sql('Impact', conn, if_exists='append')
    sleep(2)
driver.close()
```

### Economy

```python
pids_economy = pickup_economy(pids)
driver = webdriver.Chrome(chrome_path)
for p in pids_economy:
    driver.get('https://www.dotabuff.com/players/'+p+'/heroes?metric=economy')
    query = driver.find_elements_by_xpath(
        '/html/body/div[1]/div[8]/div[3]/section/article/table/tbody/*')
    rows = [tuple(i.text.split('\n')) for i in query]
    header = ['Hero', 'Date', 'GPM', 'XPM']
    economydb = pd.DataFrame(rows, columns=header)
    economydb.drop(columns='Date', inplace=True)
    economydb['PID'] = p
    economydb.to_sql('Economy', conn, if_exists='append')
    sleep(2)
driver.close()
```

### Played

```python
pids_played = pickup_played(pids)
driver = webdriver.Chrome(chrome_path)
for p in pids_played:
    driver.get('https://www.dotabuff.com/players/'+p+'/heroes?metric=played')
    query = driver.find_elements_by_xpath(
        '/html/body/div[1]/div[8]/div[3]/section/article/table/tbody/*')
    rows = [tuple(i.text.split('\n')) for i in query]
    header = ['Hero', 'Date', 'Matches',
              'Win Rate', 'KDA Ratio', 'Core', 'Safe']
    playeddb = pd.DataFrame(rows, columns=header)
    playeddb.drop(columns=['Date', 'Core', 'Safe'], inplace=True)
    playeddb['PID'] = p
    playeddb.to_sql('Played', conn, if_exists='append')
    sleep(2)
driver.close()
```

### Item Economy

```python
pids_itemeco = pickup_itemeco(pids)
driver = webdriver.Chrome(chrome_path)
for p in pids_itemeco:
    driver.get('https://www.dotabuff.com/players/'+p+'/items?metric=economy')
    query = driver.find_elements_by_xpath(
        '/html/body/div[1]/div[8]/div[3]/section/article/table/tbody/*')
    raw = [i.text.split('\n') for i in query]
    rows = [[(i, j, r[1]) for i, j in re.findall(
        r'(.*)\s(.*?\d+)$', r[0])][0] for r in raw]
    header = ['Item', 'GPM', 'XPM']
    itemecodb = pd.DataFrame(rows, columns=header)
    itemecodb['PID'] = p
    itemecodb.to_sql('ItemEco', conn, if_exists='append')
    sleep(2)
driver.close()
```

### Item Main

```python
pids_itemmain = pickup_itemmain(pids)
driver = webdriver.Chrome(chrome_path)
for p in pids_itemmain:
    driver.get('https://www.dotabuff.com/players/'+p+'/items?metric=used')
    query = driver.find_elements_by_xpath(
        '/html/body/div[1]/div[8]/div[3]/section/article/table/tbody/*')
    raw = [i.text.split('\n')[:3] for i in query]
    rows = [[(re.sub(r'\s\(level \d+\)$', '', i), j, r[1], r[2])
            for i, j in re.findall(r'(.*)\s(.*?\d+)$', r[0])][0] for r in raw]
    header = ['Item', 'Matches', 'WinRate', 'KDA_Ratio']
    itemmaindb = pd.DataFrame(rows, columns=header)
    itemmaindb['PID'] = p
    itemmaindb.to_sql('ItemMain', conn, if_exists='append')
    sleep(2)
driver.close()
```

### Cleaning up data (removing commas and % and removed heroes)

In [166]:
conn.executescript('''
            UPDATE Impact
            SET Kills = REPLACE(Kills, ',', ''),
                Death = REPLACE(Death, ',', ''),
                Assists = REPLACE(Assists, ',', '');
                
            DELETE FROM Impact
            WHERE Hero LIKE "Removed%";
                
            UPDATE Economy
            SET GPM = REPLACE(GPM, ',', ''),
                XPM = REPLACE(XPM, ',', '');
                
            DELETE FROM Economy
            WHERE Hero LIKE "Removed%";
                
            UPDATE Played
            SET Matches = REPLACE(Matches, ',', ''),
                "Win Rate" = REPLACE("Win Rate", '%', '');
                
            DELETE FROM Played
            WHERE Hero LIKE "Removed%";
                
            UPDATE ItemEco
            SET GPM = REPLACE(GPM, ',', ''),
                XPM = REPLACE(XPM, ',', '');
                
            DELETE FROM ItemEco
            WHERE Item LIKE "Removed%" OR Item LIKE "Recipe%"
            OR Item LIKE "Refresher Shard%" OR Item LIKE "River Vial%";
            
            UPDATE ItemMain
            SET Matches = REPLACE(Matches, ',', ''),
                WinRate = REPLACE("Win Rate", '%', '');
                
            DELETE FROM ItemMain
            WHERE Item LIKE "Removed%" OR Item LIKE "Recipe%"
            OR Item LIKE "Refresher Shard%" OR Item LIKE "River Vial%";
            ''')
conn.commit();

# ITAY Eto po yung tables to copy:
## Pacheck na lang po conn names

In [107]:
# pd.read_sql('''SELECT * FROM Impact''', conn).to_sql('impact', conn2)
# pd.read_sql('''SELECT * FROM Economy''', conn).to_sql('economy', conn2)
# pd.read_sql('''SELECT * FROM Played''', conn).to_sql('played', conn2)
# pd.read_sql('''SELECT * FROM ItemEco''', conn).to_sql('itemeco', conn2)
# pd.read_sql('''SELECT * FROM ItemMain''', conn).to_sql('itemmain', conn2)

# TRY TO FIX DATAFRAMES PLS.

## players

```python

# load dataframes from database
econ = pd.read_sql('''SELECT * FROM Economy''', conn)
impact = pd.read_sql('''SELECT * FROM Impact''', conn)
played = pd.read_sql('''SELECT * FROM Played''', conn)

# convert dtypes
impact['Hero'] = impact['Hero'].astype(str)
impact['PID'] = impact['PID'].astype(int)
econ['Hero'] = econ['Hero'].astype(str)
econ['PID'] = econ['PID'].astype(int)
played['Hero'] = played['Hero'].astype(str)
played['PID'] = played['PID'].astype(int)

# remove KDA Ratio
impact = impact[[i for i in impact.columns if i != 'KDA Ratio']]

# merge played and impact dataframes
df_played = played.merge(impact, left_on=['PID', 'Hero'], right_on=['PID', 'Hero'])

# clean up and merge econ with played dataframe
econ = econ[['Hero', 'GPM', 'XPM', 'PID']]
econ.drop_duplicates(inplace=True)

df_played = df_played[['Hero', 'Matches', 'Win Rate', 'KDA Ratio', 'PID',
       'Kills', 'Death', 'Assists']]

df_total = df_played.merge(econ, left_on=['PID', 'Hero'], right_on=['PID', 'Hero'])

# define function to create match probabilities
def match_prob(row):
    total = df_total[df_total['PID'] == row['PID']]['Matches'].sum()
    return row['Matches'] / total

# run match probability for df_total
df_total['match_prob'] = df_total.apply(match_prob, axis=1)

# multiply player stats by match probabilities
convert_cols = ['Win Rate', 'KDA Ratio', 'Kills', 'Death', 'Assists', 'GPM', 'XPM']
for i in convert_cols:
    df_total[i] = df_total[i] * df_total['match_prob']

# final dataframe
df_final = df_total.groupby('PID')[['Win Rate', 'KDA Ratio', 
                        'Kills', 'Death', 'Assists', 'GPM', 'XPM']].sum().reset_index()
```

In [119]:
df_played = df_played.groupby('PID')[['Matches', 'Kills', 'Death', 'Assists']].sum().reset_index()
df_played['ave_kills'] = df_played['Kills'] / df_played['Matches']
df_played['ave_death'] = df_played['Death'] / df_played['Matches']
df_played['ave_assist'] = df_played['Assists'] / df_played['Matches']
df_played = df_played.merge(df_final[['PID', 'Win Rate', 'KDA Ratio', 
                                'GPM', 'XPM']], left_on='PID', right_on='PID')

In [120]:
df_played = df_played.merge(pd.read_pickle('player_ranks.pkl'), 
                left_on='PID', right_on='players').drop('players', axis=1)

In [122]:
df_played.head()

,PID,Matches,Kills,Death,Assists,ave_kills,ave_death,ave_assist,Win Rate,KDA Ratio,GPM,XPM,rank
0,145875,1371.0,12503.0,7464.0,13464.0,9.119621,5.444201,9.820569,52.078818,3.606054,413.835157,505.447848,265
1,226583,7408.0,79220.0,54016.0,92000.0,10.693844,7.291577,12.419006,52.897262,3.262638,494.109967,550.276544,290
2,407336,4611.0,43681.0,34540.0,54302.0,9.473216,7.490783,11.776621,50.672159,2.907051,489.833550,539.688151,236
3,690740,4067.0,40505.0,27462.0,53424.0,9.959430,6.752397,13.135972,49.766988,3.532818,435.938038,497.392427,233
4,3916428,3761.0,35483.0,28229.0,46245.0,9.434459,7.505717,12.295932,60.796590,2.956741,425.391772,519.358430,17


In [129]:
# df_played.to_sql('player_summary', conn2)

## heroes

In [124]:
conn2 = sqlite3.connect('final_dota.db')

In [125]:
def hero_matchprob(row):
    total = heroes[heroes['PID'] == row['PID']]['Matches'].sum()
    return row['Matches'] / total

In [126]:
hero_cats = pd.read_sql('''SELECT * FROM hero_cat''', conn2)
heroes = played.merge(hero_cats, left_on='Hero', right_on='hero').drop(['index_x', 
                                                        'index_y'], axis=1)

heroes['match_prob'] = heroes.apply(hero_matchprob, axis=1)

for i in heroes.columns[6:-1]:
    heroes[i] = heroes[i] * heroes['match_prob']
    
player_hero = heroes.pivot_table(index='PID', columns='Hero', values='match_prob').reset_index().fillna(0)
player_herocats = heroes.groupby('PID')[heroes.columns[6:-1]].sum().reset_index().fillna(0)

hero = player_hero.merge(player_herocats)

In [127]:
hero.head()

,PID,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,...,Nukers-Pushers,Nukers-Ranged_heroes,Nukers-Strength_heroes,Nukers-Supports,Pushers-Ranged_heroes,Pushers-Strength_heroes,Pushers-Supports,Ranged_heroes-Strength_heroes,Ranged_heroes-Supports,Strength_heroes-Supports
0,145875,0.000000,0.002188,0.002918,0.037199,0.000000,0.007294,0.000000,0.006565,0.004376,...,0.196207,0.450036,0.189643,0.209336,0.196937,0.049599,0.030635,0.013129,0.167761,0.059810
1,226583,0.002438,0.009480,0.005688,0.013814,0.000271,0.009480,0.001761,0.005146,0.005688,...,0.269908,0.481853,0.194745,0.255282,0.262324,0.030200,0.068933,0.016928,0.212216,0.079496
2,407336,0.000868,0.015625,0.004557,0.006076,0.000000,0.005208,0.002821,0.004991,0.004123,...,0.241319,0.560113,0.206597,0.334418,0.214627,0.052951,0.083116,0.003472,0.276259,0.070530
3,690740,0.011065,0.012786,0.002705,0.012294,0.000492,0.029260,0.001721,0.003442,0.001967,...,0.103024,0.209737,0.366609,0.167691,0.043767,0.069830,0.009589,0.019179,0.095648,0.116548
4,3916428,0.004840,0.008067,0.008336,0.011293,0.000538,0.009680,0.005378,0.011024,0.009680,...,0.177467,0.434794,0.251681,0.321592,0.208927,0.031460,0.085776,0.041678,0.285292,0.123958


In [128]:
# hero.to_sql('hero_summary', conn2)